##Importing Libraries


In [ ]:
import os
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.preprocessing import image
import numpy as np
import matplotlib.pyplot as plt
import cv2
from PIL import Image
import imghdr
import tensorflow as tf
from tensorflow.keras.applications import InceptionV3, EfficientNetB7, ResNet101, VGG16, VGG19
from tensorflow.keras import layers, models
from tensorflow.keras import optimizers
import matplotlib.pyplot as plt
from sklearn.metrics import classification_report
import tensorflow as tf
from tf2onnx import convert
import pandas as pd
from tensorflow.keras import regularizers


##Image Augmentation Code


In [ ]:
class ImageAugmentor:
    def __init__(self, img_size=(475, 550), batch_size=32, oversample_factor=2, num_additional_images=300, output_path=None):
        self.img_size = img_size
        self.batch_size = batch_size
        self.oversample_factor = oversample_factor
        self.num_additional_images = num_additional_images
        self.output_path = output_path
        self.datagen = ImageDataGenerator(
            zoom_range=0.15,
            horizontal_flip=True,
            rotation_range=25,
            shear_range=0.1,
            featurewise_center=False,
            featurewise_std_normalization=False,
        )

    def load_minority_class_images(self, minority_class_path):
        minority_class_images = [os.path.join(minority_class_path, img) for img in os.listdir(minority_class_path)]
        return minority_class_images

    def augment_images(self, minority_class_images):
        augmented_images = []
        for img_path in minority_class_images:
            try:
                img = cv2.imread(img_path)
                img_resized = cv2.resize(img, self.img_size, interpolation=cv2.INTER_AREA)
                x = image.img_to_array(img_resized)
                x = np.expand_dims(x, axis=0)
                i = 0
                for batch in self.datagen.flow(x, batch_size=1):
                    augmented_images.append(batch[0])
                    i += 1
                    if i >= self.oversample_factor:
                        break
            except Exception as e:
                print(f"Error processing image {img_path}: {str(e)}")
        return augmented_images

    def save_augmented_images(self, augmented_images):
        if not os.path.exists(self.output_path):
            os.makedirs(self.output_path)

        for i, augmented_img in enumerate(augmented_images[:self.num_additional_images]):
            img_filename = f"augmented_image_{i + 1}.jpg"
            img_path = os.path.join(self.output_path, img_filename)
            augmented_img_rgb = cv2.cvtColor(augmented_img, cv2.COLOR_BGR2RGB)
            cv2.imwrite(img_path, augmented_img_rgb)

        print(f"{self.num_additional_images} augmented images saved to {self.output_path}")


In [ ]:
if __name__ == "__main__":
    OUTPUT_PATH = r'C:\Users\ONE BY ONE\Desktop\Image classification\images- 9 sites\petra-Treasury'
    MINORITY_CLASS_PATH = r'C:\Users\ONE BY ONE\Desktop\Image classification\images- 9 sites\petra-Treasury'
    augmentor = ImageAugmentor(
        img_size=(475, 500),
        oversample_factor=1,
        num_additional_images=300,
        output_path=OUTPUT_PATH
    )
    augmentor = ImageAugmentor(output_path=OUTPUT_PATH)
    minority_class_images = augmentor.load_minority_class_images(MINORITY_CLASS_PATH)
    augmented_images = augmentor.augment_images(minority_class_images)
    augmentor.save_augmented_images(augmented_images)

##Image Format Checking



In [ ]:

counter=[]

image_directory = r'C:\Users\ONE BY ONE\Desktop\images- 9 sites\petra-Treasury'
image_files = os.listdir(image_directory)

for image_file in image_files:
    try:
        image_path = os.path.join(image_directory, image_file)
        image_format = imghdr.what(image_path)

        if image_format:
            pass
        else:
            counter.append(image_path)
            print(f"Could not determine format for image '{image_file}'")

    except Exception as e:
        print(f"Error checking image '{image_file}': {e}")

##model


In [ ]:
## Take an indication of the average images size

links = os.listdir(r"C:\Users\ONE BY ONE\Desktop\Images\petra-Treasury")
heights=[]
widths=[]
for img in links:
    try:
        image = cv2.imread(rf"C:\Users\ONE BY ONE\Desktop\Images\petra-Treasury\{img}")
        height, width, channels = image.shape
        heights.append(height)
        widths.append(width)
    except:
        pass
np.average(heights),np.average(widths)

(475.0327056491576, 549.0049554013875)

In [ ]:
class ImageClassifier:
    def __init__(self, img_size, batch_size, num_classes, learning_rate):
        self.IMG_SIZE = img_size
        self.BATCH_SIZE = batch_size
        self.NUM_CLASSES = num_classes
        self.LEARNING_RATE = learning_rate
        self.pretrained_model = None
        self.datagen = ImageDataGenerator(
            rescale=1./255,
            shear_range=0.1,
            zoom_range=0.1,
            validation_split=0.2
        )

        self.generator = self._create_data_generator('training')
        self.validation_generator = self._create_data_generator('validation')

        self.model = None

    def _create_data_generator(self, subset):
        return self.datagen.flow_from_directory(
            r'C:\Users\ONE BY ONE\Desktop\Image classification\images- 9 sites',
            target_size=self.IMG_SIZE,
            batch_size=self.BATCH_SIZE,
            class_mode='categorical',
            subset=subset
        )

    def VGG16_build_model(self):
        base_model = self._get_pretrained_model()

        model = models.Sequential()
        model.add(base_model)
        model.add(layers.Flatten())
        model.add(layers.Dense(600, activation='relu'))
        model.add(layers.Dropout(0.5))
        model.add(layers.Dense(250, activation='relu'))
        model.add(layers.Dense(self.NUM_CLASSES, activation='softmax'))

        base_model.trainable = False

        model.compile(optimizer=optimizers.Adam(learning_rate=self.LEARNING_RATE),
                      loss='categorical_crossentropy',
                      metrics=['accuracy'])

        return model
    def InceptionV3_build_model(self):
        base_model = self._get_pretrained_model()

        model = models.Sequential()
        model.add(base_model)
        model.add(layers.GlobalAveragePooling2D())  # Change from Flatten() to GlobalAveragePooling2D()
        model.add(layers.Dense(1200, activation='relu'))
        model.add(layers.Dropout(0.5))
        model.add(layers.Dense(700, activation='relu'))
        model.add(layers.Dense(250, activation='relu'))
        model.add(layers.Dense(self.NUM_CLASSES, activation='softmax'))

        base_model.trainable = False

        model.compile(optimizer=optimizers.Adam(learning_rate=self.LEARNING_RATE),
                    loss='categorical_crossentropy',
                    metrics=['accuracy'])

        self.model = model

    def VGG19_build_model(self):
        base_model = self._get_pretrained_model()

        model = models.Sequential()
        model.add(base_model)
        model.add(layers.Flatten())
        model.add(layers.Dense(1000, activation='relu'))
        model.add(layers.Dropout(0.5))
        model.add(layers.Dense(600, activation='relu'))
        model.add(layers.Dense(250, activation='relu', kernel_regularizer=regularizers.l2(0.2)))

        model.add(layers.Dense(self.NUM_CLASSES, activation='softmax'))

        base_model.trainable = False

        model.compile(optimizer=optimizers.Adam(learning_rate=self.LEARNING_RATE),
                      loss='categorical_crossentropy',
                      metrics=['accuracy'])

        return model
    def ResNet101_build_model(self):
        base_model = self._get_pretrained_model()

        model = models.Sequential()
        model.add(base_model)
        model.add(layers.Flatten())
        model.add(layers.Dense(1400, activation='relu'))
        model.add(layers.Dropout(0.5))
        model.add(layers.Dense(800, activation='relu'))
        model.add(layers.Dense(300, activation='relu'))
        model.add(layers.Dense(self.NUM_CLASSES, activation='softmax'))

        base_model.trainable = False

        model.compile(optimizer=optimizers.Adam(learning_rate=self.LEARNING_RATE),
                      loss='categorical_crossentropy',
                      metrics=['accuracy'])
        self.model = model

    def _get_pretrained_model(self):
        if self.pretrained_model == 'InceptionV3':
            return InceptionV3(weights='imagenet', include_top=False, input_shape=(self.IMG_SIZE[0], self.IMG_SIZE[1], 3))
        elif self.pretrained_model == 'EfficientNetB7':
            return EfficientNetB7(weights='imagenet', include_top=False, input_shape=(self.IMG_SIZE[0], self.IMG_SIZE[1], 3))
        elif self.pretrained_model == 'ResNet101':
            return ResNet101(weights='imagenet', include_top=False, input_shape=(self.IMG_SIZE[0], self.IMG_SIZE[1], 3))
        elif self.pretrained_model == 'VGG16':
            return VGG16(weights='imagenet', include_top=False, input_shape=(self.IMG_SIZE[0], self.IMG_SIZE[1], 3))
        else:
            raise ValueError("Unsupported pretrained model")

    def train_model(self, epochs):
        history = self.model.fit(
            self.generator,
            epochs=epochs,
            validation_data=self.validation_generator
        )

        accuracy = self.model.evaluate(self.validation_generator)[1]
        print(f"Validation Accuracy: {accuracy * 100:.2f}%")

    def plot_learning_curve(self, history):
        # Extracting training and validation loss
        train_loss = history.history['loss']
        val_loss = history.history['val_loss']

        # Extracting training and validation accuracy (if available)
        if 'accuracy' in history.history:
            train_accuracy = history.history['accuracy']
            val_accuracy = history.history['val_accuracy']

        # Plotting training and validation loss
        plt.figure(figsize=(12, 6))
        plt.subplot(1, 2, 1)
        plt.plot(train_loss, label='Training Loss')
        plt.plot(val_loss, label='Validation Loss')
        plt.title('Training and Validation Loss')
        plt.xlabel('Epoch')
        plt.ylabel('Loss')
        plt.legend()

        # Plotting training and validation accuracy (if available)
        if 'accuracy' in history.history:
            plt.subplot(1, 2, 2)
            plt.plot(train_accuracy, label='Training Accuracy')
            plt.plot(val_accuracy, label='Validation Accuracy')
            plt.title('Training and Validation Accuracy')
            plt.xlabel('Epoch')
            plt.ylabel('Accuracy')
            plt.legend()

        plt.tight_layout()
        plt.show()

        return history

###InceptionV3_model


In [ ]:
pretrained_model_type = "InceptionV3"

img_classifier = ImageClassifier(
    img_size=(475, 550),
    batch_size=32,
    num_classes=9,
    learning_rate=0.01
)
img_classifier.pretrained_model = pretrained_model_type
img_classifier.InceptionV3_build_model()

epochs_to_train = 20
trained_history = img_classifier.train_model(epochs_to_train)


In [ ]:
img_classifier.plot_learning_curve(trained_history)

###VGG19_model


In [ ]:
pretrained_model_type = "VGG19"

img_classifier = ImageClassifier(
    img_size=(200, 250),
    batch_size=32,
    num_classes=9,
    learning_rate=0.001
)
img_classifier.pretrained_model = pretrained_model_type
img_classifier.VGG19_build_model_build_model()

epochs_to_train = 30
trained_history = img_classifier.train_model(epochs_to_train)


In [ ]:
img_classifier.plot_learning_curve(trained_history)

###VGG16_model


In [ ]:
pretrained_model_type = "VGG16"

img_classifier = ImageClassifier(
    img_size=(200, 250),
    batch_size=32,
    num_classes=9,
    learning_rate=0.001
)
img_classifier.pretrained_model = pretrained_model_type
img_classifier.VGG16_build_model_build_model()

epochs_to_train = 30
trained_history = img_classifier.train_model(epochs_to_train)



Epoch 1/20
196/196 [==============================] - 316s 2s/step - loss: 0.9971 - accuracy: 0.6750 - val_loss: 0.6513 - val_accuracy: 0.7615
Epoch 2/20
196/196 [==============================] - 320s 2s/step - loss: 0.5796 - accuracy: 0.8103 - val_loss: 0.6515 - val_accuracy: 0.7603
Epoch 3/20
196/196 [==============================] - 318s 2s/step - loss: 0.5271 - accuracy: 0.8271 - val_loss: 0.5934 - val_accuracy: 0.7801
Epoch 4/20
196/196 [==============================] - 319s 2s/step - loss: 0.4474 - accuracy: 0.8506 - val_loss: 0.6195 - val_accuracy: 0.7750
Epoch 5/20
196/196 [==============================] - 316s 2s/step - loss: 0.3682 - accuracy: 0.8737 - val_loss: 0.5523 - val_accuracy: 0.8135
Epoch 6/20
196/196 [==============================] - 318s 2s/step - loss: 0.3745 - accuracy: 0.8742 - val_loss: 0.6021 - val_accuracy: 0.7955
Epoch 7/20
196/196 [==============================] - 315s 2s/step - loss: 0.3461 - accuracy: 0.8788 - val_loss: 0.6395 - val_accuracy: 0.7974

In [ ]:
img_classifier.plot_learning_curve(trained_history)

###RESNET101_model


In [ ]:
pretrained_model_type = "ResNet101"

img_classifier = ImageClassifier(
    img_size=(475, 550),
    batch_size=32,
    num_classes=9,
    learning_rate=0.001
)
img_classifier.pretrained_model = pretrained_model_type
img_classifier.ResNet101_build_model_build_model_build_model()

epochs_to_train = 30
trained_history = img_classifier.train_model(epochs_to_train)


In [ ]:
img_classifier.plot_learning_curve(trained_history)

In [ ]:
train_accuracy_1 = np.array(history.history['accuracy'].copy())
val_accuracy_1=  np.array(history.history['val_accuracy'].copy())
train_loss_1 =  np.array(history.history['loss'].copy())
val_loss_1 =  np.array(history.history['val_loss'].copy())
train_accuracy_1,val_accuracy_1


##Classification Report



In [ ]:
model.save('C:/Users/Anas/ONE BY ONE/Desktop/Models/vgg16-30epochs.h5')

In [ ]:
generator.class_indices

{'Ajloun Castle': 0,
 'Hadrians Arch': 1,
 'Petra-siq': 2,
 'Roman Ruins-Jerash': 3,
 'Roman amphitheater': 4,
 'The Cardo Maximus of Jerash': 5,
 'Wadi Rum': 6,
 'petra-Treasury': 7,
 'umm qais': 8}

In [ ]:

true_labels = []
predicted_labels = []
for _ in range(32):
    # Get the next batch from the generator
    batch_data, batch_labels = validation_generator.next()

    # Make predictions using your model
    predictions = model.predict(batch_data)

    # Convert one-hot encoded labels to class indices
    true_labels.extend(np.argmax(batch_labels, axis=1))
    predicted_labels.extend(np.argmax(predictions, axis=1))


1/1 [==============================] - 1s 1s/step


In [ ]:
# class_names = generator.class_indices.keys()
print(classification_report(true_labels, predicted_labels, target_names=class_names))



                             precision    recall  f1-score   support

              Ajloun Castle       0.87      0.97      0.92        67
              Hadrians Arch       0.87      0.84      0.86        77
                  Petra-siq       0.99      0.84      0.91        96
         Roman Ruins-Jerash       0.60      0.48      0.53       136
         Roman amphitheater       0.57      0.85      0.68       120
The Cardo Maximus of Jerash       0.91      0.80      0.85       139
                   Wadi Rum       0.87      0.96      0.91       135
             petra-Treasury       0.91      0.98      0.94       125
                   umm qais       0.87      0.67      0.75       129

                   accuracy                           0.81      1024
                  macro avg       0.83      0.82      0.82      1024
               weighted avg       0.82      0.81      0.81      1024



In [ ]:


# Load the TensorFlow model from the h5 file
model_path = r"C:\Users\ONE BY ONE\Desktop\Models\vgg19-30epochs.h5"
modelt = tf.keras.models.load_model(model_path)

# Convert the TensorFlow model to ONNX format
onnx_model, _ = convert.from_keras(modelt)

# Save the ONNX model to a file
onnx_path = r"C:\Users\ONE BY ONE\Desktop\Models\vgg16-20epochs.onnx"
with open(onnx_path, 'wb') as f:
    f.write(onnx_model.SerializeToString())
print(f'Model converted and saved to: {onnx_path}')


Model converted and saved to: C:\Users\ONE BY ONE\Desktop\Models\vgg16-20epochs.onnx
